# Brightway2 seminar
Chris Mutel ([PSI](https://www.psi.ch/)), Pascal Lesage ([CIRAIG](http://www.ciraig.org/en/))

## Day 1, afternoon  
Session on working with `Activity` and ` Exchange` proxies 

### Learning objectives  
  - Work with `Activity` and ` Exchange` proxies (copy, add, modify, delete)  
  - (OPTIONAL - CHRIS?) Calculate uncertainty based on Pedigree  

**Exercise**: Create a new activity for offset printing in Quebec (CA-QC) that is based on a copy of the activity `'offset printing, per kg printed paper' (kilogram, CH, None)`. It is to have the following modifications:  
  - Its location is to be changed to "CA-QC"  
  - Its source of electricity is to be changed to electricity from the market for low-voltage electricity in CA-QC  
  - Suppose the printing generates no waste. Its paper input needs to be set to 1kg, and its waste paper output needs to be deleted. 

CHRIS: this may be fun, but probably would take too long.
  - The Pedigree matrix scores for the exchanges are to be modified  
  - The uncertainty is to be modified based on these modified Pedigree scores

**Step 1**: Import Brightway as bw, open the seminar project and make a copy of the ecoinvent v3.3 database (this step can be long, let's do it before the break).

In [1]:
import brightway2 as bw

In [2]:
bw.projects.set_current('bw2_seminar_2017')

In [3]:
if 'ecoinvent 3.3 cutoff copy' not in bw.databases:
    bw.Database('ecoinvent 3.3 cutoff').copy('ecoinvent 3.3 cutoff copy')
else:
    print('A copy already exists')
eidb = bw.Database('ecoinvent 3.3 cutoff copy')

Writing activities to SQLite3 database:
0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:21:24


Title: Writing activities to SQLite3 database:
  Started: 03/27/2017 16:27:47
  Finished: 03/27/2017 16:49:11
  Total time elapsed: 00:21:24
  CPU %: 12.20
  Memory %: 22.40


**Step 2**: Create a copy of the activity `'offset printing, per kg printed paper' (kilogram, CH, None)` and assign it to a variable.  

In [4]:
# Solution
CH_printing = [act for act in eidb if "offset printing" in act['name'] if act['location']=="CH"][0]
QC_printing = CH_printing.copy()

**Step 3**: Just a check: make sure that the key of the activities are not the same

In [5]:
CH_printing.key == QC_printing.key

False

**Step 4**: Change the QC activity's location to "CA-QC"

In [6]:
QC_printing['location']="CA-QC"
QC_printing.save()

**Step 5**: Find the electricity exchange to be replaced in the Quebec printing activity.

In [7]:
electricity_input_exchange = [exc for exc in QC_printing.technosphere() if "electricity" in exc['name']][0]
electricity_input_exchange

Exchange: 0.719 kilowatt hour 'market for electricity, low voltage' (kilowatt hour, CH, None) to 'offset printing, per kg printed paper' (kilogram, CA-QC, None)>

**Step 6**: Find the activity that this exchange needs to be relinked to.

In [8]:
qc_elect=[act for act in eidb 
                     if 'electricity' in act['name']
                     if 'low voltage' in act['name']
                     if 'market' in act['activity type']
                     if act['location']=='CA-QC'
                     ][0]

In [9]:
qc_elect

'market for electricity, low voltage' (kilowatt hour, CA-QC, None)

**Step 7**: Relink the electricity input exchange to the QC electricity:

In [10]:
electricity_input_exchange['input'] = qc_elect.key
electricity_input_exchange.save()

**Step 8**: Delete the waste paper output.

In [11]:
waste_paper_output = [exc for exc in QC_printing.technosphere() if "paper" in exc['name'] if exc['amount']<0][0]
waste_paper_output

Exchange: -0.0882 kilogram 'market for waste paperboard' (kilogram, CH, None) to 'offset printing, per kg printed paper' (kilogram, CA-QC, None)>

In [12]:
waste_paper_output.delete()

**Step 9**: Change the paper input amount to 1 kg.

In [13]:
paper_input = [exc for exc in QC_printing.technosphere() if "paper, woodfree" in exc['name']][0]
paper_input

Exchange: 1.39 kilogram 'market for paper, woodfree, coated' (kilogram, RER, None) to 'offset printing, per kg printed paper' (kilogram, CA-QC, None)>

In [14]:
paper_input['amount']=1
paper_input.save()

In [15]:
QC_printing.save()

**Final result:** 

In [16]:
for exc in QC_printing.exchanges():
    print(bw.get_activity(exc['input'])['name'], exc['amount'], exc['unit'])

market for white spirit 0.00305 kilogram
market for spent solvent mixture -0.0015 kilogram
offset printing, per kg printed paper 1.0 kilogram
market for waste emulsion paint -0.000282 kilogram
market for corrugated board box 0.0882 kilogram
market for ethylene glycol monoethyl ether 0.000524 kilogram
market for printing ink, offset, without solvent, in 47.5% solution state 0.00927 kilogram
market for acrylic dispersion, without water, in 65% solution state 0.00119 kilogram
market for isopropanol 0.00218 kilogram
market for industrial machine, heavy, unspecified 0.000218 kilogram
market for sheet rolling, aluminium 0.0249 kilogram
market for thermoforming, with calendering 0.000398 kilogram
market for heat, district or industrial, natural gas 1.58 megajoule
market for synthetic rubber 0.000179 kilogram
market for paper, woodfree, coated 1 kilogram
market for acrylic varnish, without water, in 87.5% solution state 0.000391 kilogram
market for electricity, low voltage 0.719 kilowatt hour


In [17]:
for exc in CH_printing.exchanges():
    print(bw.get_activity(exc['input'])['name'], exc['amount'], exc['unit'])

market for white spirit 0.00305 kilogram
market for spent solvent mixture -0.0015 kilogram
offset printing, per kg printed paper 1.0 kilogram
market for waste emulsion paint -0.000282 kilogram
market for corrugated board box 0.0882 kilogram
market for ethylene glycol monoethyl ether 0.000524 kilogram
market for waste paperboard -0.0882 kilogram
market for printing ink, offset, without solvent, in 47.5% solution state 0.00927 kilogram
market for acrylic dispersion, without water, in 65% solution state 0.00119 kilogram
market for isopropanol 0.00218 kilogram
market for industrial machine, heavy, unspecified 0.000218 kilogram
market for sheet rolling, aluminium 0.0249 kilogram
market for thermoforming, with calendering 0.000398 kilogram
market for heat, district or industrial, natural gas 1.58 megajoule
market for synthetic rubber 0.000179 kilogram
market for paper, woodfree, coated 1.39 kilogram
market for acrylic varnish, without water, in 87.5% solution state 0.000391 kilogram
market f